In [3]:
from pulp import *
import pandas as pd

data = pd.read_csv("data/cluster.csv", header=0)


# only existing vertiport and total profit columns
clusterName = data['label']
dataTable = data.iloc[:, -3:].values.tolist()
print(dataTable)

n_Ve = dict([(i, n[0]) for i, n in enumerate(dataTable)])
total_commute = dict([(i, n[1]) for i, n in enumerate(dataTable)])
total_prof = dict([(i, n[2]) for i, n in enumerate(dataTable)])

prob = LpProblem('MaxProfit', LpMinimize)

# number of existing vertiport
n_eVTOL_Vars = LpVariable.dicts('n_eVTOL', clusterName, 0, cat='Integer')
# number of new vertiport
n_Vn_Vars = LpVariable.dicts('n_Vn', clusterName, 0, cat='Integer')

n_round = 15
c_eVTOL = 1000000.0 ## updated to 1 million
c_Ve = 200000.0
c_Vn = 500000.0
cap_eVTOL = 4.0
cap_port = 8.0
depreciation_1 = 1/365
depreciation_10 = 1/365/10
depreciation_20 = 1/365/20

# Objection function
prob += lpSum([- total_prof[i] + c_eVTOL * n_eVTOL_Vars[i]*depreciation_10 + c_Ve * n_Ve[i]*depreciation_1 + c_Vn *
               n_Vn_Vars[i] * depreciation_20 for i in clusterName]), 'Total Profit'

# Constraint
for i in clusterName:
    prob += cap_eVTOL * n_eVTOL_Vars[i] * n_round >= total_commute[i]
    prob += cap_port * (n_Ve[i] + n_Vn_Vars[i]) >= n_eVTOL_Vars[i]

prob.solve()

print()

for var in prob.variables():
    print(str(var) + ' : ' + str(var.varValue))
print()

print("Total profit = $%.2f" % value(-prob.objective))
# res_dict = {'new_vertiport': [var.varValue[:10] for var in prob.variables()],
#             'eVTOL#': [var.varValue[10:] for var in prob.variables()]}
# res = pd.DataFrame(res_dict)
res = [var.varValue for var in prob.variables()]
res_df = pd.DataFrame(columns=['new_vertiport', 'eVTOL#'])
res_df['new_vertiport'] = res[:10]
res_df['eVTOL#'] = res[10:]
#res_df.to_csv('data/optimization_result.csv', index=False)

[[22, 2398, 23418], [4, 1948, 26239], [18, 2994, 36338], [3, 1210, 26221], [14, 2301, 25531], [12, 2317, 23761], [19, 1516, 23336], [5, 1469, 29907], [13, 2925, 33054], [49, 2224, 20626]]

n_Vn_0 : 0
n_Vn_1 : 1
n_Vn_2 : 0
n_Vn_3 : 0
n_Vn_4 : 0
n_Vn_5 : 0
n_Vn_6 : 0
n_Vn_7 : 0
n_Vn_8 : 0
n_Vn_9 : 0
n_eVTOL_0 : 40
n_eVTOL_1 : 33
n_eVTOL_2 : 50
n_eVTOL_3 : 21
n_eVTOL_4 : 39
n_eVTOL_5 : 39
n_eVTOL_6 : 26
n_eVTOL_7 : 25
n_eVTOL_8 : 49
n_eVTOL_9 : 38

Total profit = $82609.08


In [2]:
res_df['eVTOL#'].sum()

360

In [7]:
from pulp import *
import pandas as pd

data = pd.read_csv("data/cluster.csv", header=0)


# only existing vertiport and total profit columns
clusterName = data['label']
dataTable = data.iloc[:, -3:].values.tolist()
print(dataTable)

n_Ve = dict([(i, n[0]) for i, n in enumerate(dataTable)])
total_commute = dict([(i, n[1]) for i, n in enumerate(dataTable)])
total_prof = dict([(i, n[2]) for i, n in enumerate(dataTable)])

prob = LpProblem('MaxProfit', LpMinimize)

# number of existing vertiport
n_eVTOL_Vars = LpVariable.dicts('n_eVTOL', clusterName, 0, cat='Integer')
# number of new vertiport
n_Vn_Vars = LpVariable.dicts('n_Vn', clusterName, 0, cat='Integer')

n_round = 15
c_eVTOL = 1000000.0 ## updated to 1 million
c_Ve = 200000.0
c_Vn = 500000.0
cap_eVTOL = 4.0
cap_port = 8.0
depreciation_1 = 1/365
depreciation_10 = 1/365/10
depreciation_20 = 1/365/20

# Objection function
prob += lpSum([- total_prof[i] + c_eVTOL * n_eVTOL_Vars[i]*depreciation_10 + c_Ve * n_Ve[i]*depreciation_1 + c_Vn *
               n_Vn_Vars[i] * depreciation_20 for i in clusterName]), 'Total Profit'

# Constraint
for i in clusterName:
    prob += cap_eVTOL * n_eVTOL_Vars[i] * n_round >= total_commute[i]
    prob += cap_port * (n_Ve[i] + n_Vn_Vars[i]) >= n_eVTOL_Vars[i]

prob.solve()

print()

for var in prob.variables():
    print(str(var) + ' : ' + str(var.varValue))
print()

print("Total profit = $%.2f" % value(-prob.objective))
# res_dict = {'new_vertiport': [var.varValue[:10] for var in prob.variables()],
#             'eVTOL#': [var.varValue[10:] for var in prob.variables()]}
# res = pd.DataFrame(res_dict)
res = [var.varValue for var in prob.variables()]
res_df = pd.DataFrame(columns=['new_vertiport', 'eVTOL#'])
res_df['new_vertiport'] = res[:10]
res_df['eVTOL#'] = res[10:]
#res_df.to_csv('data/optimization_result.csv', index=False)

TypeError: LpVariable.dicts() missing 1 required positional argument: 'name'

In [13]:
data

,label,geometry,pop,centroid,pop_0,dist_0,eVTOL%_0,pop_1,dist_1,eVTOL%_1,...,eVTOL%_7,pop_8,dist_8,eVTOL%_8,pop_9,dist_9,eVTOL%_9,existing_vertiport,total_commute,total_prof
0,0,POINT (147524.8267484695 -437785.3796935198),1214421.0,POINT (-118.4017715911054 34.06434931240423),0.0,0.000000,0.00125,197877.2,29.318110,0.00125,...,0.00125,295633.5,11.472608,0.00125,225528.2,12.474179,0.00125,22,2398,23418
1,1,POINT (193831.06946395693 -428732.802066179),764351.0,POINT (-117.89807531601825 34.137793434079526),197877.2,29.318110,0.00125,0.0,0.000000,0.00125,...,0.00125,250626.5,28.812345,0.00125,180521.2,17.328543,0.00125,4,1948,26239
2,2,POINT (167030.82056639862 -458331.7134100427),1810280.0,POINT (-118.19473570919335 33.87598418360855),302470.1,17.604002,0.00125,257463.1,24.810978,0.00125,...,0.00125,355219.4,7.661317,0.00125,285114.1,17.708769,0.00125,18,2994,36338
3,3,POINT (143242.84805150024 -375789.90918792645),26843.0,POINT (-118.43708102592144 34.62362212208346),124126.4,38.614072,0.00125,79119.4,45.500941,0.00125,...,0.00125,176875.7,49.498010,0.00125,106770.4,36.439536,0.00125,3,1210,26221
4,4,POINT (181964.84146615295 -441425.06661846023),1117462.0,POINT (-118.0295766456479 34.02570119732147),233188.3,21.519260,0.00125,188181.3,10.796536,0.00125,...,0.00125,285937.6,18.461113,0.00125,215832.3,12.269456,0.00125,14,2301,25531
5,5,POINT (144718.57185065473 -430795.4029925484),1133076.0,POINT (-118.43091491784193 34.12777392511734),234749.7,4.680337,0.00125,189742.7,30.544068,0.00125,...,0.00125,287499.0,16.148277,0.00125,217393.7,13.215592,0.00125,12,2317,23761
6,6,POINT (124913.0147257182 -415583.7872378607),332484.0,POINT (-118.64326141611919 34.26762080145649),154690.5,19.690799,0.00125,109683.5,43.596261,0.00125,...,0.00125,207439.8,30.695761,0.00125,137334.5,27.006041,0.00125,19,1516,23336
7,7,POINT (181357.9910229577 -380749.8981071864),285889.0,POINT (-118.02244317128229 34.5725295741208),150031.0,41.206567,0.00125,105024.0,30.806159,0.00125,...,0.00125,202780.3,48.620550,0.00125,132675.0,31.974726,0.00125,5,1469,29907
8,8,POINT (155301.56650390083 -454531.02827989496),1741914.0,POINT (-118.32075059793034 33.91219003709535),295633.5,11.472608,0.00125,250626.5,28.812345,0.00125,...,0.00125,0.0,0.000000,0.00125,278277.5,16.705293,0.00125,13,2925,33054
9,9,POINT (165966.02700243032 -429852.1820274433),1040861.0,POINT (-118.20039910815179 34.132871192047006),225528.2,12.474179,0.00125,180521.2,17.328543,0.00125,...,0.00125,278277.5,16.705293,0.00125,0.0,0.000000,0.00125,49,2224,20626
